In [10]:
import pandas as pd

df = pd.read_csv('soccer_data.csv')
df.head()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater1,rater2,refNum,refCountry,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.25,0.50,1,1,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.75,0.75,2,2,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,...,NaN,NaN,3,3,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,...,NaN,NaN,3,3,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,...,NaN,NaN,3,3,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002


### Transformations

In [11]:
# let's say we want to add winrate of the club
def calc_win_rate(group):
    group['club_win_rate'] = group['victories'].sum() / group['games'].sum()
    return group

df = df.groupby('club', group_keys=False).apply(calc_win_rate).reset_index(drop=True)

In [12]:
# Here I have alternatives for how I want to specify the skin tone rating of the players.
# Because I will use the column in the same way later, I will repeat the same column name. 

df['rater_combined'] = (df['rater1'] + df['rater2']) / 2
# take the max of the two raters
df['rater_combined'] = df[['rater1', 'rater2']].max(axis=1)
# take the min of the two raters
df['rater_combined'] = df[['rater1', 'rater2']].min(axis=1)

# do this only for when we take the max
df = df[df['rater_combined'] < 1.0]

In [13]:
# now I want to group the dataframe by the player and I pick the variables I want to keep
df = df.groupby('playerShort').agg(
    games_total=("games", "sum"),
    victories_total=("victories", "sum"),
    rating=("rater_combined", "first"),
    red_cards_total=("redCards", "sum"),
    yellow_cards_total=("yellowCards", "sum"),
    weight_player=("weight", "first"),
    height_player=("height", "first"),
    win_rate_club=("club_win_rate", "first")
).reset_index()
df.head()


,playerShort,games_total,victories_total,rating,red_cards_total,yellow_cards_total,weight_player,height_player,win_rate_club
0,aaron-hughes,654,247,0.00,0,19,71.0,182.0,0.425597
1,aaron-hunt,336,141,0.00,1,42,73.0,183.0,0.403489
2,aaron-lennon,412,200,0.25,0,11,63.0,165.0,0.463656
3,aaron-ramsey,260,150,0.00,1,31,76.0,178.0,0.523065
4,abdelhamid-el-kaoutari,124,41,0.25,2,8,73.0,180.0,0.402834


In [14]:
# I then consturct some more variables based on this new dataframe
df['player_win_rate'] = df['games_total'] / df['victories_total']
df['bmi'] = df['weight_player'] / (df['height_player'] / 100) ** 2

### Statistical Model

In [15]:
import statsmodels.formula.api as smf

model = smf.ols(formula='rating ~ red_cards_total + rating + player_win_rate', data=df)
model.fit().summary()


PatsyError: Error evaluating factor: NameError: name 'rater_combined' is not defined
    rater_combined ~ red_cards_total + rating + player_win_rate
    ^^^^^^^^^^^^^^